## Description:
这次实验依然是用的Criteo数据集， 只不过由于原来的数据量太大， 为了在单机上能够运行， 做了采样， 取了很少的一部分进行实验。数据集位于data/文件夹下， train.csv是训练集， test.csv是测试集。 这个笔记本我们是做数据的读入和预处理操作， 具体步骤如下：
1. 读入数据集， 并进行缺失值的填充， 这里为了简单一些， 直接类别特征填充“-1”， 数值特征填充0
2. 类别特征的编码， 用的LabelEncoder编码， 数值特征的归一化处理
3. 划分开训练集和验证集保存到prepeocessed/文件夹下

## 导入包和数据集

In [1]:
# import packages
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')


In [8]:
train_df.head()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000743,1,1.0,0,1.0,NaN,227.0,1.0,173.0,18.0,...,3486227d,e88ffc9d,c393dc22,b1252a9d,57c90cd9,NaN,bcdee96c,4d19a3eb,cb079c2d,456c12a0
1,10000159,1,4.0,1,1.0,2.0,27.0,2.0,4.0,2.0,...,07c540c4,92555263,NaN,NaN,242bb710,NaN,3a171ecb,72c78f11,NaN,NaN
2,10001166,1,0.0,806,NaN,NaN,1752.0,142.0,2.0,0.0,...,07c540c4,25c88e42,21ddcdc9,b1252a9d,a0136dd2,NaN,32c7478e,8fc66e78,001f3601,f37f3967
3,10000318,0,2.0,-1,42.0,14.0,302.0,38.0,25.0,38.0,...,e5ba7672,5aed7436,21ddcdc9,b1252a9d,c3abeb21,NaN,423fab69,1793a828,e8b83407,5cef228f
4,10000924,1,0.0,57,2.0,1.0,2891.0,2.0,35.0,1.0,...,e5ba7672,642f2610,1d1eb838,b1252a9d,1640d50b,ad3062eb,423fab69,45ab94c8,2bf691b1,c84c4aec


In [10]:
test_df.head()

,Id,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000405,NaN,-1,NaN,NaN,8020.0,26.0,6.0,0.0,80.0,...,e5ba7672,7119e567,1d04f4a4,b1252a9d,d5f54153,NaN,32c7478e,a9d771cd,c9f3bea7,0a47000d
1,10001189,NaN,-1,NaN,NaN,17881.0,9.0,8.0,0.0,0.0,...,e5ba7672,51369abb,NaN,NaN,d4b6b7e8,NaN,32c7478e,37821b83,NaN,NaN
2,10000674,0.0,0,2.0,13.0,2904.0,104.0,1.0,3.0,100.0,...,e5ba7672,bd17c3da,966f1c31,a458ea53,1d1393f4,ad3062eb,32c7478e,3fdb382b,010f6491,49d68486
3,10001358,0.0,1471,51.0,4.0,1573.0,63.0,1.0,4.0,13.0,...,d4bb7bd8,1f9656b8,21ddcdc9,b1252a9d,602ce342,NaN,3a171ecb,1793a828,e8b83407,70b6702c
4,10000810,0.0,16,9.0,17.0,2972.0,621.0,13.0,42.0,564.0,...,e5ba7672,87c6f83c,NaN,NaN,bf8efd4c,c9d4222a,423fab69,f96a556f,NaN,NaN


In [12]:
test_df.head().append(test_df.tail())

,Id,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000405,NaN,-1,NaN,NaN,8020.0,26.0,6.0,0.0,80.0,...,e5ba7672,7119e567,1d04f4a4,b1252a9d,d5f54153,NaN,32c7478e,a9d771cd,c9f3bea7,0a47000d
1,10001189,NaN,-1,NaN,NaN,17881.0,9.0,8.0,0.0,0.0,...,e5ba7672,51369abb,NaN,NaN,d4b6b7e8,NaN,32c7478e,37821b83,NaN,NaN
2,10000674,0.0,0,2.0,13.0,2904.0,104.0,1.0,3.0,100.0,...,e5ba7672,bd17c3da,966f1c31,a458ea53,1d1393f4,ad3062eb,32c7478e,3fdb382b,010f6491,49d68486
3,10001358,0.0,1471,51.0,4.0,1573.0,63.0,1.0,4.0,13.0,...,d4bb7bd8,1f9656b8,21ddcdc9,b1252a9d,602ce342,NaN,3a171ecb,1793a828,e8b83407,70b6702c
4,10000810,0.0,16,9.0,17.0,2972.0,621.0,13.0,42.0,564.0,...,e5ba7672,87c6f83c,NaN,NaN,bf8efd4c,c9d4222a,423fab69,f96a556f,NaN,NaN
395,10001453,1.0,0,1.0,NaN,149.0,5.0,1.0,0.0,0.0,...,d4bb7bd8,5aed7436,d16737e3,a458ea53,edc49a33,NaN,93bad2c0,3fdb382b,e8b83407,80dd0a5b
396,10000360,NaN,-1,NaN,NaN,NaN,NaN,0.0,0.0,6.0,...,2005abd1,5162930e,NaN,NaN,12965bb8,NaN,32c7478e,71292dbb,NaN,NaN
397,10001809,0.0,300,4.0,NaN,4622.0,25.0,20.0,6.0,55.0,...,8efede7f,a78bd508,21ddcdc9,5840adea,c2a93b37,NaN,3a171ecb,1793a828,e8b83407,2fede552
398,10000769,1.0,1,2.0,1.0,5.0,1.0,1.0,1.0,1.0,...,d4bb7bd8,a1d0cc4f,c68db44a,a458ea53,3b1ae854,NaN,32c7478e,57e2c6c9,1575c75f,7132fed8
399,10000563,NaN,2,NaN,NaN,36144.0,NaN,NaN,36.0,NaN,...,e5ba7672,7b49e3d2,NaN,NaN,dfcfc3fa,NaN,423fab69,aee52b6f,NaN,NaN


In [14]:
test_df.shape

(400, 40)

In [15]:
train_df.shape

(1599, 41)

In [16]:
test_df.describe()

,Id,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13
count,4.000000e+02,206.000000,400.00000,308.000000,314.000000,381.000000,299.000000,384.000000,400.000000,384.000000,206.000000,384.000000,84.000000,314.000000
mean,1.000099e+07,4.490291,121.06750,23.064935,8.630573,16134.677165,152.498328,14.906250,13.187500,137.317708,0.587379,2.153646,0.976190,13.175159
std,6.046310e+02,10.281758,366.27015,82.450394,10.033673,59475.506157,422.632350,56.557109,22.934085,379.675529,0.624411,3.753479,4.415133,28.854453
min,1.000000e+07,0.000000,-1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000046e+07,0.000000,0.00000,2.000000,2.000000,295.000000,11.000000,0.000000,2.000000,10.750000,0.000000,0.000000,0.000000,2.000000
50%,1.000098e+07,1.000000,3.00000,7.000000,4.000000,2904.000000,40.000000,3.000000,7.000000,42.500000,1.000000,1.000000,0.000000,5.000000
75%,1.000153e+07,4.750000,55.00000,22.000000,12.000000,10764.000000,138.500000,8.250000,19.000000,124.500000,1.000000,2.000000,0.000000,15.000000
max,1.000200e+07,88.000000,2974.00000,1361.000000,51.000000,681386.000000,4638.000000,825.000000,390.000000,5637.000000,3.000000,32.000000,39.000000,386.000000


In [17]:
train_df.describe()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13
count,1.599000e+03,1599.000000,904.000000,1599.000000,1242.000000,1265.000000,1.525000e+03,1207.000000,1522.000000,1597.000000,1522.000000,904.000000,1522.000000,361.000000,1265.000000
mean,1.000100e+07,0.208881,3.515487,120.161976,35.562802,7.921739,1.431446e+04,145.545153,13.408673,13.795241,124.099869,0.555310,2.157687,1.058172,11.439526
std,5.703227e+02,0.406636,9.504431,441.761111,269.324202,9.632292,5.288257e+04,313.674504,52.673756,22.957518,244.009440,0.634234,3.492093,5.478184,25.962510
min,1.000000e+07,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000051e+07,0.000000,0.000000,0.000000,2.000000,2.000000,2.990000e+02,9.500000,1.000000,2.000000,10.000000,0.000000,0.000000,0.000000,2.000000
50%,1.000100e+07,0.000000,1.000000,2.000000,6.000000,4.000000,2.875000e+03,43.000000,3.000000,8.000000,43.000000,0.000000,1.000000,0.000000,5.000000
75%,1.000149e+07,0.000000,3.000000,40.000000,19.000000,11.000000,1.026200e+04,140.000000,10.750000,21.000000,129.000000,1.000000,2.000000,1.000000,14.000000
max,1.000200e+07,1.000000,95.000000,7864.000000,8457.000000,87.000000,1.015215e+06,3156.000000,1658.000000,547.000000,3387.000000,4.000000,37.000000,98.000000,770.000000


In [18]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 40 columns):
Id     400 non-null int64
I1     206 non-null float64
I2     400 non-null int64
I3     308 non-null float64
I4     314 non-null float64
I5     381 non-null float64
I6     299 non-null float64
I7     384 non-null float64
I8     400 non-null float64
I9     384 non-null float64
I10    206 non-null float64
I11    384 non-null float64
I12    84 non-null float64
I13    314 non-null float64
C1     400 non-null object
C2     400 non-null object
C3     380 non-null object
C4     380 non-null object
C5     400 non-null object
C6     346 non-null object
C7     400 non-null object
C8     400 non-null object
C9     400 non-null object
C10    400 non-null object
C11    400 non-null object
C12    380 non-null object
C13    400 non-null object
C14    400 non-null object
C15    400 non-null object
C16    380 non-null object
C17    400 non-null object
C18    400 non-null object
C19    214 non-null ob

In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 41 columns):
Id       1599 non-null int64
Label    1599 non-null int64
I1       904 non-null float64
I2       1599 non-null int64
I3       1242 non-null float64
I4       1265 non-null float64
I5       1525 non-null float64
I6       1207 non-null float64
I7       1522 non-null float64
I8       1597 non-null float64
I9       1522 non-null float64
I10      904 non-null float64
I11      1522 non-null float64
I12      361 non-null float64
I13      1265 non-null float64
C1       1599 non-null object
C2       1599 non-null object
C3       1553 non-null object
C4       1553 non-null object
C5       1599 non-null object
C6       1402 non-null object
C7       1599 non-null object
C8       1599 non-null object
C9       1599 non-null object
C10      1599 non-null object
C11      1599 non-null object
C12      1553 non-null object
C13      1599 non-null object
C14      1599 non-null object
C15      1599 non

In [32]:
train_df.shape[0]

1599

In [3]:
def sparsFeature(feat, feat_num, embed_dim=4):
    """
    create dictionary for sparse feature
    :param feat: feature_name
    :param feat_num: the total number of sparse features that do not repeat
    :param embed_dim: embedding dimension
    :return
    """
    return {'feat': feat, 'feat_num': feat_num, 'embed_dim': embed_dim}

def denseFeature(feat):
    """
    create dictionary for dense feature
    :param feat: dense feature name
    : return
    """
    return {'feat': feat}

In [4]:
# 读入数据集，并进行预处理
def create_cretio_data(embed_dim=8, test_size=0.2):
    # import data
    train_df = pd.read_csv('./data/train.csv')
    test_df = pd.read_csv('./data/test.csv')
    
    # 进行数据合并
    label = train_df['Label']
    del train_df['Label']

    data_df = pd.concat((train_df, test_df))
    del data_df['Id']
    
    print(data_df.columns)
    # 特征分开类别
    # 根据列名包含值
    sparse_feas = [col for col in data_df.columns if col[0] == 'C']
    dense_feas = [col for col in data_df.columns if col[0] == 'I']
    
    # 填充缺失值
    data_df[sparse_feas] = data_df[sparse_feas].fillna('-1')
    data_df[dense_feas] = data_df[dense_feas].fillna(0)
    
    # 把特征列保存成字典, 方便类别特征的处理工作
    # 合并两个特征list
    feature_columns = [[denseFeature(feat) for feat in dense_feas]] + [[sparsFeature(feat, len(data_df[feat].unique()), embed_dim=embed_dim) for feat in sparse_feas]]
    np.save('preprocessed_data/fea_col.npy', feature_columns)
    
    
    # 数据预处理
    # 进行编码  类别特征编码
    for feat in sparse_feas:
        le = LabelEncoder()
        data_df[feat] = le.fit_transform(data_df[feat])
    
    # 数值特征归一化
    mms = MinMaxScaler()
    data_df[dense_feas] = mms.fit_transform(data_df[dense_feas])
    
    # 分开测试集和训练集
    train = data_df[:train_df.shape[0]]
    test = data_df[train_df.shape[0]:]

    train['Label'] = label
    
    # 划分验证集
    train_set, val_set = train_test_split(train, test_size = 0.2, random_state=2020)
    
    # 保存文件
    train_set.reset_index(drop=True, inplace=True)
    val_set.reset_index(drop=True, inplace=True)

    train_set.to_csv('preprocessed_data/train_set.csv', index=0)
    val_set.to_csv('preprocessed_data/val_set.csv', index=0)
    test.to_csv('preprocessed_data/test_set.csv', index=0)

In [22]:
 sparse_feas = [col for col in train_df.columns if col[0] == 'C']


In [23]:
sparse_feas

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 'C23',
 'C24',
 'C25',
 'C26']

In [25]:
dense_feas = [col for col in train_df.columns if col[0] == 'I']
dense_feas

['Id',
 'I1',
 'I2',
 'I3',
 'I4',
 'I5',
 'I6',
 'I7',
 'I8',
 'I9',
 'I10',
 'I11',
 'I12',
 'I13']

In [29]:
embed_dim = 8
feature_columns = [[denseFeature(feat) for feat in dense_feas]] + [[sparsFeature(feat, len(train_df[feat].unique()), embed_dim=embed_dim) for feat in sparse_feas]]

In [30]:
feature_columns

[[{'feat': 'Id'},
  {'feat': 'I1'},
  {'feat': 'I2'},
  {'feat': 'I3'},
  {'feat': 'I4'},
  {'feat': 'I5'},
  {'feat': 'I6'},
  {'feat': 'I7'},
  {'feat': 'I8'},
  {'feat': 'I9'},
  {'feat': 'I10'},
  {'feat': 'I11'},
  {'feat': 'I12'},
  {'feat': 'I13'}],
 [{'feat': 'C1', 'feat_num': 73, 'embed_dim': 8},
  {'feat': 'C2', 'feat_num': 230, 'embed_dim': 8},
  {'feat': 'C3', 'feat_num': 1054, 'embed_dim': 8},
  {'feat': 'C4', 'feat_num': 854, 'embed_dim': 8},
  {'feat': 'C5', 'feat_num': 30, 'embed_dim': 8},
  {'feat': 'C6', 'feat_num': 7, 'embed_dim': 8},
  {'feat': 'C7', 'feat_num': 986, 'embed_dim': 8},
  {'feat': 'C8', 'feat_num': 34, 'embed_dim': 8},
  {'feat': 'C9', 'feat_num': 2, 'embed_dim': 8},
  {'feat': 'C10', 'feat_num': 766, 'embed_dim': 8},
  {'feat': 'C11', 'feat_num': 799, 'embed_dim': 8},
  {'feat': 'C12', 'feat_num': 1011, 'embed_dim': 8},
  {'feat': 'C13', 'feat_num': 713, 'embed_dim': 8},
  {'feat': 'C14', 'feat_num': 19, 'embed_dim': 8},
  {'feat': 'C15', 'feat_num': 

In [20]:
create_cretio_data()

Index(['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11',
       'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
       'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
       'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'],
      dtype='object')


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
